### Music gereration firth Keras and TensorFlow backend

Plan was simple:
1. Read midi file, convert it to matrix of features
2. Create simple model with Keras and LSTM to learn the pattern
3. Use subsample of initial midi file as a input for model to generate pure art
4. Save prediction from model to midi file
.
.
.
5. PROFIT

<i> For disclamer: I've been using my old Dell Laptop with no GPU support</i>

In [2]:
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import numpy as np

/Users/kamalsdu/main/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


##### Read midi file

In [3]:
mid = MidiFile('Samples/Nintendo_-_Pokemon_Fire_Red_Route_1_Piano_Cover_Hard_Version.mid') 
notes = []
velocities = []

##### Extract notes and velocities and then compine it in the one list of lists

In [4]:
for msg in mid:
    if not msg.is_meta:
        if msg.channel == 0:
            if msg.type == 'note_on':
                data = msg.bytes()
                # append note and velocity from [type, note, velocity]
                note = data[1]
                velocity = data[2]
                notes.append(note)
                velocities.append(velocity)
combine = [[i,j] for i,j in zip(notes, velocities) ]

#### Apply min-max scalling

In [5]:
note_min = np.min(notes)
note_max = np.max(notes)
velocities_min = np.min(velocities)
velocities_max = np.max(velocities)

for i in combine:
    i[0] = 2*(i[0]-((note_min+note_max)/2))/(note_max-note_min)
    i[1] = 2*(i[1]-((velocities_min+velocities_max)/2))/(velocities_max-velocities_min)


C:\Users\home\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


#### Prepare features for training and data subsample for prediction

In [18]:
X = []
Y = []
n_prev = 30
for i in range(len(combine)-n_prev):
    x = combine[i:i+n_prev]
    y = combine[i+n_prev]
    X.append(x)
    Y.append(y)
# save a seed to do prediction later
seed = combine[0:n_prev]

#### Made sequential model with several layers, use LSTM as it time dependent data

I also whant to save checkpoints

In [22]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 2), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, input_shape=(n_prev, 2), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(64, input_shape=(n_prev, 2), return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(2))
model.add(Activation('linear'))
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)
filepath = 'check/checkpoint_model_{epoch:02d}.hdf5'
model_save_callback = ModelCheckpoint(filepath, monitor='val_acc', 
                                      verbose=1, save_best_only=False, 
                                      mode='auto', period=5)

#### Train your model.
It might take a while, I was waiting for 1 hour with just 5 epoch

In [36]:
model.fit(np.array(X), np.array(Y), 50, 100, verbose=1, callbacks=[model_save_callback])


Epoch 1/100
13070/13070 [==============================] - 34s 3ms/step - loss: nan
Epoch 2/100
13070/13070 [==============================] - 34s 3ms/step - loss: nan
Epoch 3/100
13070/13070 [==============================] - 34s 3ms/step - loss: nan
Epoch 4/100
13070/13070 [==============================] - 34s 3ms/step - loss: nan

Epoch 00004: saving model to check/checkpoint_model_04.hdf5
Epoch 5/100
13070/13070 [==============================] - 34s 3ms/step - loss: nan
Epoch 6/100
13070/13070 [==============================] - 37s 3ms/step - loss: nan
Epoch 7/100
13070/13070 [==============================] - 36s 3ms/step - loss: nan
Epoch 8/100
13070/13070 [==============================] - 37s 3ms/step - loss: nan
Epoch 9/100
13070/13070 [==============================] - 37s 3ms/step - loss: nan

Epoch 00009: saving model to check/checkpoint_model_09.hdf5
Epoch 10/100
13070/13070 [==============================] - 36s 3ms/step - loss: nan
Epoch 11/100
13070/13070 [===========

KeyboardInterrupt: 

#### Make a prediction

In [34]:
prediction = []
x = seed
x = np.expand_dims(x, axis=0)

for i in range(300):
    preds = model.predict(x)
    x = np.squeeze(x)
    x = np.concatenate((x, preds))
    x = x[1:]
    x = np.expand_dims(x, axis=0)
    preds = np.squeeze(preds)
    prediction.append(preds)



In [35]:
pred

array([nan, nan], dtype=float32)

#### Reverse the min-max scalling

In [32]:
for pred in prediction:
# Undo the preprocessing
    pred[0] = int((pred[0]/2)*(note_max-note_min) + (note_min+note_max)/2)
    pred[1] = int((pred[1]/2)*(velocities_max-velocities_min) + (velocities_min+velocities_max)/2)
    if pred[0] < 24:
        pred[0] = 24
    elif pred[0] > 102:
        pred[0] = 102
    if pred[1] < 0:
        pred[1] = 0
    elif pred[1] > 127:
        pred[1] = 127


ValueError: cannot convert float NaN to integer

#### Save your result to new midi file

In [33]:
mid = MidiFile()
track = MidiTrack()

t = 0
for note in prediction:
    # 147 means note_on
    note = np.asarray([147, note[0], note[1]])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)

mid.tracks.append(track)
mid.save('Generated_song_epoch =10.mid')

ValueError: data byte must be in range 0..127

#### Just listen to it. The result is surreal!